In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ML_utils import load_data,load_data_test_set, total_day_eve_night_grouping
df =pd.read_csv('train.csv')

### UNCOMMENT THIS PART TO GROUP FEATURES
feature_engineering = True
df=total_day_eve_night_grouping(df,grouping=feature_engineering)
exclude_list = [feature_name for feature_name in df.columns if feature_name not in['number_vmail_messages','number_customer_service_calls','total_charges','churn']]
one_hot_on = True
normalize_on = True
oversample_on=True



In [74]:

###################
#### LOAD DATA ####

X_train,y_train,X_val,y_val,X_test,y_test,columns = load_data(df,exclude=exclude_list,one_hot=one_hot_on,normalize_=normalize_on,oversample=oversample_on)


y_train[y_train==0]=-1

### kernel not trick

# X_train=np.concatenate((X_train,X_train**3),axis=1)

# X_val=np.concatenate((X_val,X_val**3),axis=1)
# X_train.shape
# X_train=X_train.astype(np.double)
# y_train=y_train.astype(np.double)

# indicies_0=np.where(y_train==0)[0]

# indicies_1=np.where(y_train==1)[0]
# N=indicies_0.shape[0]
# M=indicies_1.shape[0]

# X0=X_train[indicies_0]
# X1=X_train[indicies_1]

# X = np.concatenate((X0.T, X1.T), axis = 1)
# y = np.concatenate((np.ones((1, N)), -1*np.ones((1, M))), axis = 1)

In [75]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import GridSearchCV

# # clf = SVC(kernel='poly',degree=3,coef0=1)
# clf= SVC(kernel='poly',C=1)

# clf.fit(X_train, y_train.reshape(-1,))

# clf.predict(X_val)

# print(accuracy_score(y_val,clf.predict(X_val)))

In [76]:
def kernel(x,z,sigma=4):
    n = x.shape[0]
    m = z.shape[0]
    xx = np.dot(np.sum(np.power(x, 2), 1).reshape(n, 1), np.ones((1, m)))
    zz = np.dot(np.sum(np.power(z, 2), 1).reshape(m, 1), np.ones((1, n)))     
    return np.exp(-(xx + zz.T - 2 * np.dot(x, z.T)) / (2 * sigma ** 2))

In [77]:
import numpy as np
from cvxopt import matrix, solvers

X=X_train.astype(np.double)
y=y_train.astype(np.double)
C = 0.8
m, n = X.shape
K = kernel(X,X)
P = matrix(np.matmul(y,y.T) * K)
q = matrix(np.ones((m, 1)) * -1)
A = matrix(y.reshape(1, -1))
b = matrix(np.zeros(1))          
G = matrix(np.vstack((np.eye(m) * -1, np.eye(m))))        
h = matrix(np.hstack((np.zeros(m), np.ones(m) * C)))

In [78]:
solution = solvers.qp(P, q, G, h, A, b)



     pcost       dcost       gap    pres   dres
 0: -1.5386e+03 -5.5722e+03  3e+04  3e+00  3e-14
 1: -9.9206e+02 -3.9986e+03  5e+03  4e-01  2e-14
 2: -9.1533e+02 -1.6497e+03  8e+02  3e-02  1e-14
 3: -9.8892e+02 -1.2434e+03  3e+02  8e-03  1e-14
 4: -1.0189e+03 -1.1589e+03  1e+02  4e-03  1e-14
 5: -1.0418e+03 -1.1029e+03  6e+01  1e-03  1e-14
 6: -1.0500e+03 -1.0856e+03  4e+01  7e-04  1e-14
 7: -1.0553e+03 -1.0757e+03  2e+01  3e-04  1e-14
 8: -1.0586e+03 -1.0697e+03  1e+01  2e-04  1e-14
 9: -1.0596e+03 -1.0681e+03  9e+00  1e-04  1e-14
10: -1.0614e+03 -1.0652e+03  4e+00  4e-05  1e-14
11: -1.0624e+03 -1.0637e+03  1e+00  7e-06  1e-14
12: -1.0628e+03 -1.0632e+03  3e-01  1e-06  1e-14
13: -1.0629e+03 -1.0630e+03  1e-01  2e-07  1e-14
14: -1.0630e+03 -1.0630e+03  1e-02  2e-08  2e-14
15: -1.0630e+03 -1.0630e+03  8e-04  9e-10  2e-14
Optimal solution found.


In [79]:
alphas = np.array(solution['x'])
ind = (alphas > 1e-6).flatten()
sv = X[ind]
sv_y = y[ind]
alphas = alphas[ind]

In [80]:
b = sv_y - np.sum(kernel(sv,sv) * alphas * sv_y, axis=0)
b = np.sum(b) / b.size

In [81]:
prod = np.sum(kernel(sv,X_val) * alphas * sv_y, axis=0) + b
predictions = np.sign(prod)

In [82]:
predictions.reshape(-1,1)
predictions[predictions==-1]=0
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val,predictions))

0.8398744113029827


In [83]:
def predict(X):
    prod = np.sum(kernel(sv,X) * alphas * sv_y, axis=0) + b
    predictions = np.sign(prod) 
    predictions[predictions==-1]=0
    return predictions

In [84]:
### USE THE MODEL ON THE TEST SET
df_test=pd.read_csv('test.csv')

df_test=total_day_eve_night_grouping(df_test,grouping=feature_engineering)


X_test_output=load_data_test_set(df_test,exclude=exclude_list+['id'],one_hot=one_hot_on,normalize_=normalize_on)

y_pred_output = predict(X_test_output)

y_pred_output=np.where(y_pred_output>0.5,'yes', 'no')
y_pred_output=y_pred_output.reshape(-1,)
id_column = np.arange(1, y_pred_output.shape[0] + 1)


# Create a DataFrame
df_output = pd.DataFrame({
    'id': id_column,
    'churn': y_pred_output
})


# Save the DataFrame as a CSV file
df_output.to_csv('output_svm.csv', index=False)